# Prédiction des données lidar avec les données physiques

#### Librairies 

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np


from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error, max_error, mean_absolute_percentage_error

import math

from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import PowerTransformer
import h5py
from keras.models import load_model

import matplotlib.pyplot as plt
import seaborn as sns

#### Données 

In [19]:
df = pd.read_excel('./df_cleaned.xlsx')

Index(['fractal_dimension', 'fraction_of_coating (%)',
       'primary_particle_size (nm)', 'number_of_primary_particles',
       'vol_equi_radius_outer (nm)', 'vol_equi_radius_inner (nm)',
       'equi_mobility_dia (nm)', 'mie_epsilon', 'length_scale_factor',
       'm_real_bc', 'm_im_bc', 'm_real_organics', 'm_im_organics',
       'volume_total (nm^3)', 'volume_bc (nm^3)', 'volume_organics (nm^3)',
       'density_bc (g/cm^3)', 'density_organics (g/cm^3)', 'mass_bc (g)',
       'mass_organics (g)', 'mass_total  (g)', 'mr_total/bc', 'mr_nonBC/BC',
       'MEC_530', 'MEC_467', 'Cbac_530', 'Cbac_467', 'MBC_530', 'MBC_467',
       'LR_530', 'LR_467', 'CR', 'BAE'],
      dtype='object')

In [20]:
X = df.iloc[:, :23]  # données particules
Y = df.iloc[:,23:31]  # données optiques
L = df.iloc[:,31:]  # données Lidar

In [21]:
print(X.columns)
print(Y.columns)
print(L.columns)

Index(['fractal_dimension', 'fraction_of_coating (%)',
       'primary_particle_size (nm)', 'number_of_primary_particles',
       'vol_equi_radius_outer (nm)', 'vol_equi_radius_inner (nm)',
       'equi_mobility_dia (nm)', 'mie_epsilon', 'length_scale_factor',
       'm_real_bc', 'm_im_bc', 'm_real_organics', 'm_im_organics',
       'volume_total (nm^3)', 'volume_bc (nm^3)', 'volume_organics (nm^3)',
       'density_bc (g/cm^3)', 'density_organics (g/cm^3)', 'mass_bc (g)',
       'mass_organics (g)', 'mass_total  (g)', 'mr_total/bc', 'mr_nonBC/BC'],
      dtype='object')
Index(['MEC_530', 'MEC_467', 'Cbac_530', 'Cbac_467', 'MBC_530', 'MBC_467',
       'LR_530', 'LR_467'],
      dtype='object')
Index(['CR', 'BAE'], dtype='object')


In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(
            X, L,
            test_size=0.30,
            random_state=10)

pt = PowerTransformer(method='yeo-johnson')

X_test=X_test.reset_index(drop=True)
Y_test=Y_test.reset_index(drop=True)


X_train_transformed = scaler.fit_transform(X_train)  #pt.fit_transform(X_train)
X_test_transformed =  scaler.fit_transform(X_test)   #pt.transform(X_test)

Y_train_transformed = pd.DataFrame(pt.fit_transform(Y_train), columns=Y_train.columns)
Y_test_transformed = pd.DataFrame(pt.transform(Y_test), columns=Y_test.columns)

In [25]:
para  = X.columns
print(para)
print(Y_test.columns)

Index(['fractal_dimension', 'fraction_of_coating (%)',
       'primary_particle_size (nm)', 'number_of_primary_particles',
       'vol_equi_radius_outer (nm)', 'vol_equi_radius_inner (nm)',
       'equi_mobility_dia (nm)', 'mie_epsilon', 'length_scale_factor',
       'm_real_bc', 'm_im_bc', 'm_real_organics', 'm_im_organics',
       'volume_total (nm^3)', 'volume_bc (nm^3)', 'volume_organics (nm^3)',
       'density_bc (g/cm^3)', 'density_organics (g/cm^3)', 'mass_bc (g)',
       'mass_organics (g)', 'mass_total  (g)', 'mr_total/bc', 'mr_nonBC/BC'],
      dtype='object')
Index(['CR', 'BAE'], dtype='object')


Nombre de lignes :

In [26]:
k = 3883

## Régression linéaire 

### Définition
La régression linéaire est une méthode statistique et machine learning utilisée pour modéliser la relation entre une **variable cible** (\(Y\)) et une ou plusieurs **variables explicatives** (\(X\)) à l'aide d'une fonction linéaire.

---

### Modèle mathématique
Le modèle de régression linéaire est défini par :

$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p + \epsilon $
- $Y$ : Variable cible (dépendante).
- $X_1, X_2, \dots, X_p$ : Variables explicatives (indépendantes).
- $\beta_0$ : Intercept (ordonnée à l'origine).
- $\beta_1, \beta_2, \dots, \beta_p$ : Coefficients des variables.
- $\epsilon$ : Terme d'erreur (résiduel).

---

### Objectif
- Trouver les coefficients $\beta_0, \beta_1, \dots, \beta_p$ qui minimisent l'erreur entre les prédictions du modèle $\hat{Y}$ et les valeurs réelles $Y$.
- Cette minimisation est généralement réalisée par la **méthode des moindres carrés**, qui minimise la somme des carrés des résidus :
$ \text{Erreur} = \sum_{i=1}^{n} (Y_i - \hat{Y}_i)^2 $

In [27]:
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import pandas as pd
from joblib import dump  # Importation de joblib pour sauvegarder les modèles

# Initialiser le modèle de régression linéaire
linear_model = LinearRegression()

# Initialiser un dictionnaire pour stocker les métriques et les prédictions
results_linear = []
Y_pred_linear = pd.DataFrame()

# Créer le dossier pour les modèles si il n'existe pas
output_dir = 'Best_models/X_L/Linear'
os.makedirs(output_dir, exist_ok=True)

# Boucle sur chaque variable cible
for i, col in enumerate(Y_train.columns):
    # Ajuster le modèle sur la colonne actuelle (transformée)
    linear_model.fit(X_train_transformed[:k, :], Y_train_transformed.iloc[:k, i])
    
    # Prédire les valeurs sur le jeu de test (transformé)
    y_pred = linear_model.predict(X_test_transformed[:k, :])
    y_true = Y_test.iloc[:k, i]
    
    # Pas de transformation inverse ici : on utilise directement y_pred (transformation appliquée)
    y_pred_original = y_pred  # Les prédictions sont déjà dans l'échelle transformée

    # Calculer les métriques
    mse = mean_squared_error(y_true, y_pred_original)
    mape = mean_absolute_percentage_error(y_true, y_pred_original)
    r2 = r2_score(y_true, y_pred_original)
    
    # Stocker les résultats
    results_linear.append({
        'Target Column': col,
        'MSE': mse,
        'MAPE': mape,
        'R2': r2
    })
    
    # Stocker les prédictions dans un DataFrame
    Y_pred_linear[col] = y_pred_original

    # Définir le chemin d'enregistrement du modèle pour chaque colonne
    model_path = os.path.join(output_dir, f'{col}_best_model_linear.joblib')  # Sauvegarder dans le bon dossier
    dump(linear_model, model_path)
    print(f"Modèle pour la variable '{col}' enregistré sous {model_path}")

    # Afficher les métriques pour la variable cible
    print(f"Target Column '{col}'")
    print(f"MSE: {mse}, MAPE: {mape}, R²: {r2}")
    print("-" * 40)

# Créer un DataFrame pour résumer les résultats
params_linear = pd.DataFrame(results_linear)

# Afficher le tableau des résultats
print(params_linear)

# Afficher les prédictions (si nécessaire)
# print(Y_pred_linear)


Modèle pour la variable 'CR' enregistré sous Best_models/X_L/Linear\CR_best_model_linear.joblib
Target Column 'CR'
MSE: 220651076206.2078, MAPE: 492488.4962253989, R²: -11696358993438.053
----------------------------------------
Modèle pour la variable 'BAE' enregistré sous Best_models/X_L/Linear\BAE_best_model_linear.joblib
Target Column 'BAE'
MSE: 202523403570.62344, MAPE: 128366.3746839029, R²: -25175063667.090736
----------------------------------------
  Target Column           MSE           MAPE            R2
0            CR  2.206511e+11  492488.496225 -1.169636e+13
1           BAE  2.025234e+11  128366.374684 -2.517506e+10


## Random split (KRR)

### Définition
La **régression à noyau** (KRR) combine deux concepts puissants :
1. **Régression ridge** : Une variante de la régression linéaire qui ajoute une pénalité pour limiter la complexité du modèle.
2. **Trick du noyau** : Une technique permettant de modéliser des relations non linéaires en projetant les données dans un espace de caractéristiques de dimension supérieure.

---

### Modèle mathématique
La KRR minimise la fonction coût suivante :

$ \text{Erreur} = ||Y - K \alpha||^2 + \lambda ||\alpha||^2 $
- $Y$ : Cibles réelles.
- $K$ : Matrice de noyau, définie par $K_{ij} = k(X_i, X_j)$, où $k(\cdot, \cdot)$ est une fonction de noyau.
- $\alpha$ : Coefficients à déterminer.
- $\lambda$ : Hyperparamètre de régularisation qui contrôle le compromis biais/variance.

---

### Fonction de noyau
La fonction de noyau $k(X_i, X_j)$ mesure la similarité entre deux observations. Les noyaux courants sont :
- **Linéaire** : $k(X_i, X_j) = X_i^T X_j$
- **RBF (Radial Basis Function)** : $k(X_i, X_j) = \exp\left(-\frac{||X_i - X_j||^2}{2\sigma^2}\right)$

Le choix du noyau permet de capturer des relations linéaires ou non linéaires.

In [28]:
import os
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
from joblib import dump  # Importation de joblib pour sauvegarder les modèles

# Paramètres pour GridSearchCV pour Kernel Ridge Regression
param_grid = {
    'alpha': [0.1, 1, 10],           # Paramètre de régularisation
    'kernel': ['linear', 'rbf'],     # Choix de noyaux
    'gamma': [0.1, 1, 10]            # Paramètre pour le noyau 'rbf'
}

# Initialiser un dictionnaire pour stocker les meilleurs modèles pour chaque variable cible
best_models = {}
best_params_list = []

# Créer le dossier pour les modèles si il n'existe pas
output_dir = 'Best_models/X_L/KRR'
os.makedirs(output_dir, exist_ok=True)

# Boucle sur chaque variable cible (chaque colonne de Y_train)
for i, col in enumerate(Y_train.columns):
    # Initialiser un modèle Kernel Ridge
    krr = KernelRidge()

    # Configurer la recherche de grille
    grid_search = GridSearchCV(
        estimator=krr,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_absolute_percentage_error',  # Utiliser MAPE pour optimiser
        n_jobs=-1
    )

    # Ajuster le modèle sur la colonne actuelle de Y_train
    grid_search.fit(X_train_transformed[:k, :], Y_train.iloc[:k, i])

    # Obtenir le meilleur modèle
    best_model = grid_search.best_estimator_
    best_models[col] = best_model

    # Prédire sur les données de test
    y_pred = best_model.predict(X_test_transformed[:k, :])
    y_true = Y_test.iloc[:k, i]

    # Calculer les erreurs
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Stocker les résultats
    best_params_list.append({
        'Target Column': col,
        'alpha': grid_search.best_params_['alpha'],
        'kernel': grid_search.best_params_['kernel'],
        'gamma': grid_search.best_params_['gamma'],
        'MSE': mse,
        'MAPE': mape,
        'R2': r2
    })

    # Sauvegarder le modèle pour chaque colonne dans le dossier spécifié
    model_path = os.path.join(output_dir, f'{col}_best_model_KRR.joblib')  # Sauvegarder dans le bon dossier
    dump(best_model, model_path)
    print(f"Modèle pour la variable '{col}' enregistré sous {model_path}")

    # Afficher les résultats pour chaque colonne
    print(f"Target Column '{col}'")
    print("Best Parameters:", grid_search.best_params_)
    print(f"MSE: {mse}, MAPE: {mape}, R²: {r2}")
    print("-" * 40)

# Créer un DataFrame pour résumer les résultats
params_KRR = pd.DataFrame(best_params_list)

# Afficher le tableau des résultats
print(params_KRR)

# Créer un DataFrame pour les prédictions
Y_pred_KRR = pd.DataFrame()
for column, model in best_models.items():
    Y_pred_KRR[column] = model.predict(X_test_transformed[:k, :])

# Afficher les prédictions
#print(Y_pred_KRR)


Modèle pour la variable 'CR' enregistré sous Best_models/X_L/KRR\CR_best_model_KRR.joblib
Target Column 'CR'
Best Parameters: {'alpha': 0.1, 'gamma': 1, 'kernel': 'rbf'}
MSE: 0.007280196439357656, MAPE: 0.10234298566648692, R²: 0.6140884850345986
----------------------------------------
Modèle pour la variable 'BAE' enregistré sous Best_models/X_L/KRR\BAE_best_model_KRR.joblib
Target Column 'BAE'
Best Parameters: {'alpha': 0.1, 'gamma': 1, 'kernel': 'rbf'}
MSE: 2.173436776075444, MAPE: 0.44331952066411257, R²: 0.7298267348287547
----------------------------------------
  Target Column  alpha kernel  gamma       MSE      MAPE        R2
0            CR    0.1    rbf      1  0.007280  0.102343  0.614088
1           BAE    0.1    rbf      1  2.173437  0.443320  0.729827


## Gradient boosting 

### Définition
Le **Gradient Boosting** est une technique d'ensemble qui construit un modèle puissant en combinant plusieurs modèles faibles (souvent des arbres de décision) de manière séquentielle. À chaque étape, le modèle suivant corrige les erreurs du modèle précédent en optimisant une fonction de perte grâce à la descente de gradient.

---

### Modèle mathématique
L'objectif est de minimiser une fonction de perte $L(Y, \hat{Y})$, où :
- $Y$ : Cibles réelles.
- $\hat{Y}$ : Prédictions du modèle.

---

### Hyperparamètres clés
- **Nombre d'estimateurs** $(n\_estimators)$ : Nombre total d'arbres.
- **Taux d'apprentissage** $(learning\_rate)$ : Contrôle la contribution de chaque arbre.
- **Profondeur maximale** $(max\_depth)$ : Limite la complexité des arbres.

In [29]:
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
from joblib import dump  # Importation de joblib pour sauvegarder les modèles

# Paramètres pour GridSearchCV pour GradientBoostingRegressor
param_grid = {
    'n_estimators': [10, 100, 300, 500],
    'learning_rate': [0.05, 0.1, 0.5],
    'max_depth': [2, 3]
}

# Initialiser un dictionnaire pour stocker les meilleurs modèles pour chaque variable cible
best_models = {}
best_params_list = []

# DataFrame pour stocker les prédictions pour chaque variable cible
Y_pred_GB = pd.DataFrame()

# Créer le dossier pour les modèles si il n'existe pas
output_dir = 'Best_models/X_L/GB'
os.makedirs(output_dir, exist_ok=True)

# Boucle sur chaque variable de sortie (chaque colonne de Y_train)
for i, col in enumerate(Y_train.columns):
    # Initialiser un modèle de GradientBoostingRegressor
    gbr = GradientBoostingRegressor()
    
    # Configurer la recherche de grille
    grid_search = GridSearchCV(
        estimator=gbr,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_absolute_percentage_error',
        n_jobs=-1
    )
    
    # Ajuster le modèle sur la colonne actuelle de Y_train
    grid_search.fit(X_train_transformed[:k, :], Y_train.iloc[:k, i])
    
    # Enregistrer le meilleur modèle pour la variable cible actuelle
    best_model = grid_search.best_estimator_
    best_models[col] = best_model
    
    # Prédictions sur l'ensemble de test
    y_pred = best_model.predict(X_test_transformed)
    y_true = Y_test.iloc[:, i]
    
    # Ajouter les prédictions au DataFrame Y_pred_GB
    Y_pred_GB[col] = y_pred
    
    # Calcul des métriques
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Ajouter les meilleurs paramètres et les scores dans la liste des meilleurs paramètres
    best_params_list.append({
        'Variable': col,
        'n_estimators': grid_search.best_params_['n_estimators'],
        'learning_rate': grid_search.best_params_['learning_rate'],
        'max_depth': grid_search.best_params_['max_depth'],
        'Best Score (MAPE Negatif)': grid_search.best_score_,
        'MSE': mse,
        'MAPE': mape,
        'R2': r2
    })
    
    # Sauvegarder le modèle pour chaque colonne dans le dossier spécifié
    model_path = os.path.join(output_dir, f'{col}_best_model_GBR.joblib')  # Sauvegarder dans le bon dossier
    dump(best_model, model_path)
    print(f"Modèle pour la variable '{col}' enregistré sous {model_path}")

    # Afficher les meilleurs hyperparamètres et les scores pour la variable cible actuelle
    print(f"Variable de sortie '{col}'")
    print("Meilleurs paramètres :", grid_search.best_params_)
    print("Meilleur score (MAPE négatif) :", grid_search.best_score_)
    print(f"MSE: {mse}, MAPE: {mape}, R²: {r2}")
    print("-" * 40)

# Créer un DataFrame pour stocker les meilleurs paramètres et les métriques de chaque modèle
params_GB = pd.DataFrame(best_params_list)

# Afficher le DataFrame des meilleurs paramètres
#print(params_GB)

Modèle pour la variable 'CR' enregistré sous Best_models/X_L/GB\CR_best_model_GBR.joblib
Variable de sortie 'CR'
Meilleurs paramètres : {'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 300}
Meilleur score (MAPE négatif) : -0.04168073607851962
MSE: 0.010432305222432388, MAPE: 0.20795463195796818, R²: 0.4470002634536647
----------------------------------------
Modèle pour la variable 'BAE' enregistré sous Best_models/X_L/GB\BAE_best_model_GBR.joblib
Variable de sortie 'BAE'
Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
Meilleur score (MAPE négatif) : -0.13382454959911722
MSE: 5.438318139802836, MAPE: 0.5764805803787817, R²: 0.32397933768120646
----------------------------------------


## XGBoost 

### Définition
XGBoost est une implémentation avancée et optimisée de la méthode Gradient Boosting. Elle est conçue pour être :
- **Rapide** grâce à des optimisations matérielles et algorithmiques.
- **Précise** avec des techniques intégrées de régularisation.

---

### Modèle mathématique

L'objectif est de minimiser une fonction de perte régulière définie par : 

$ \mathcal{L}(\Theta) = \sum_{i=1}^{n} L(Y_i, \hat{Y}_i) + \sum_{k=1}^{K} \Omega(f_k) $
- $L(Y_i, \hat{Y}_i)$ : Fonction de perte
- $\Omega(f_k)$ : Terme de régularisation pour éviter le surapprentissage.

$ \Omega(f_k) = \gamma T + \frac{1}{2} \lambda ||w||^2 $
  - $T$ : Nombre de feuilles dans l'arbre.
  - $w$ : Poids des feuilles.
  - $\gamma, \lambda$ : Hyperparamètres de régularisation.

---

### Hyperparamètres clés
- **Nombre d'estimateurs** $(n\_estimators)$ : Nombre total d'arbres.
- **Taux d'apprentissage** $(learning\_rate)$ : Contrôle la contribution de chaque arbre.
- **Profondeur maximale** $(max\_depth)$ : Limite la complexité des arbres.
- **Subsample** et **ColSampleByTree** : Contrôle du sous-échantillonnage.


In [30]:
import os
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import pandas as pd
from joblib import dump  # Importation de joblib pour sauvegarder les modèles

# Paramètres pour GridSearchCV pour XGBoost
param_grid = {
    'n_estimators': [100, 300, 500],  # Nombre d'arbres
    'learning_rate': [0.05, 0.1, 0.3],  # Taux d'apprentissage
    'max_depth': [3, 5, 7],  # Profondeur maximale des arbres
    'subsample': [0.8, 1],  # Fraction des échantillons utilisés pour entraîner chaque arbre
    'colsample_bytree': [0.8, 1]  # Fraction des caractéristiques utilisées pour chaque arbre
}

# Initialiser un dictionnaire pour stocker les meilleurs modèles pour chaque variable cible
best_models = {}
best_params_list = []

# Créer le dossier pour les modèles si il n'existe pas
output_dir = 'Best_models/X_L/XGB'
os.makedirs(output_dir, exist_ok=True)

# Boucle sur chaque variable cible (chaque colonne de Y_train)
for i, col in enumerate(Y_train.columns):
    # Initialiser un modèle XGBoost
    xgbr = XGBRegressor(objective='reg:squarederror', n_jobs=-1)  # Configuré pour minimiser l'erreur quadratique

    # Configurer la recherche de grille
    grid_search = GridSearchCV(
        estimator=xgbr,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_absolute_percentage_error',  # Optimisation avec MAPE
        n_jobs=-1
    )

    # Ajuster le modèle sur la colonne actuelle de Y_train
    grid_search.fit(X_train_transformed[:k, :], Y_train.iloc[:k, i])

    # Obtenir le meilleur modèle
    best_model = grid_search.best_estimator_
    best_models[col] = best_model

    # Prédire sur les données de test
    y_pred = best_model.predict(X_test_transformed[:k, :])
    y_true = Y_test.iloc[:k, i]

    # Calculer les erreurs
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Stocker les résultats
    best_params_list.append({
        'Target Column': col,
        'n_estimators': grid_search.best_params_['n_estimators'],
        'learning_rate': grid_search.best_params_['learning_rate'],
        'max_depth': grid_search.best_params_['max_depth'],
        'subsample': grid_search.best_params_['subsample'],
        'colsample_bytree': grid_search.best_params_['colsample_bytree'],
        'MSE': mse,
        'MAPE': mape,
        'R2': r2
    })

    # Sauvegarder le modèle pour chaque colonne dans le dossier spécifié
    model_path = os.path.join(output_dir, f'{col}_best_model_XGB.joblib')  # Sauvegarder dans le bon dossier
    dump(best_model, model_path)
    print(f"Modèle pour la variable '{col}' enregistré sous {model_path}")

    # Afficher les résultats pour chaque colonne
    print(f"Target Column '{col}'")
    print("Best Parameters:", grid_search.best_params_)
    print(f"MSE: {mse}, MAPE: {mape}, R²: {r2}")
    print("-" * 40)

# Créer un DataFrame pour résumer les résultats
params_XGB = pd.DataFrame(best_params_list)

# Afficher le tableau des résultats
print(params_XGB)

# Créer un DataFrame pour les prédictions
Y_pred_XGB = pd.DataFrame()
for column, model in best_models.items():
    Y_pred_XGB[column] = model.predict(X_test_transformed[:k, :])

# Afficher les prédictions (si nécessaire)
# print(Y_pred_XGB)

Modèle pour la variable 'CR' enregistré sous Best_models/X_L/XGB\CR_best_model_XGB.joblib
Target Column 'CR'
Best Parameters: {'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 500, 'subsample': 0.8}
MSE: 0.04624872366978813, MAPE: 0.26213562194069123, R²: -1.4515705263303182
----------------------------------------
Modèle pour la variable 'BAE' enregistré sous Best_models/X_L/XGB\BAE_best_model_XGB.joblib
Target Column 'BAE'
Best Parameters: {'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 500, 'subsample': 0.8}
MSE: 12.741706877760201, MAPE: 0.7348624062777284, R²: -0.5838825352147683
----------------------------------------
  Target Column  n_estimators  learning_rate  max_depth  subsample  \
0            CR           500           0.05          7        0.8   
1           BAE           500           0.05          7        0.8   

   colsample_bytree        MSE      MAPE        R2  
0                 1   0.046249  0.262136 -1

## ANN 

### Définition
Les **réseaux de neurones artificiels** (ANN) sont des modèles inspirés du cerveau humain, capables de détecter des motifs complexes dans les données. Ils sont constitués de couches de **neurones** interconnectés, organisées en trois types principaux de couches :
- **Entrée** : Reçoit les données d'entrée.
- **Cachées** : Effectuent les transformations et calculs complexes.
- **Sortie** : Produit les prédictions finales.

---

### Modèle mathématique

Chaque neurone dans une couche effectue un calcul basé sur une somme pondérée des entrées, suivie d'une activation non linéaire :

$ z = \sum_{i=1}^{n} w_i x_i + b $

$a = \phi(z)$
- $w_i$ : Poids associés aux entrées.
- $x_i$ : Entrées.
- $b$ : Biais.
- $\phi$ : Fonction d'activation (ex. ReLU, sigmoïde, tanh).

Les poids et les biais sont ajustés pendant l'entraînement pour minimiser une fonction de perte.

---

### Hyperparamètres clés

- **hidden_layer_sizes** : Détermine la structure des couches cachées. Chaque tuple dans cette liste représente le nombre de neurones dans chaque couche cachée.
- **activation** : Fonction d'activation à utiliser dans les couches cachées.
- **learning_rate_init** : Taux d'apprentissage initial, contrôlant la vitesse à laquelle le modèle ajuste ses poids.
- **max_iter** : Nombre maximal d'itérations (ou d'époques) pendant l'entraînement.

In [31]:
import os
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
from joblib import dump  # Importation de joblib pour sauvegarder les modèles

# Paramètres pour GridSearchCV pour MLPRegressor (ANN)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],  # Architectures des couches cachées
    'activation': ['relu', 'tanh'],                               # Fonctions d'activation
    'learning_rate_init': [0.001, 0.01],                          # Taux d'apprentissage initial
    'max_iter': [500, 1000]                                       # Nombre maximal d'itérations
}

# Initialiser un dictionnaire pour stocker les meilleurs modèles pour chaque variable cible
best_models = {}
best_params_list = []

# Créer le dossier pour les modèles si il n'existe pas
output_dir = 'Best_models/X_L/ANN'
os.makedirs(output_dir, exist_ok=True)

# Boucle sur chaque variable cible (chaque colonne de Y_train)
for i, col in enumerate(Y_train.columns):
    # Initialiser un modèle ANN (MLPRegressor)
    mlp = MLPRegressor(random_state=42)

    # Configurer la recherche de grille
    grid_search = GridSearchCV(
        estimator=mlp,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_absolute_percentage_error',  # Optimisation avec MAPE
        n_jobs=-1
    )

    # Ajuster le modèle sur la colonne actuelle de Y_train
    grid_search.fit(X_train_transformed[:k, :], Y_train.iloc[:k, i])

    # Obtenir le meilleur modèle
    best_model = grid_search.best_estimator_
    best_models[col] = best_model

    # Prédire sur les données de test
    y_pred = best_model.predict(X_test_transformed[:k, :])
    y_true = Y_test.iloc[:k, i]

    # Calculer les erreurs
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Stocker les résultats
    best_params_list.append({
        'Target Column': col,
        'hidden_layer_sizes': grid_search.best_params_['hidden_layer_sizes'],
        'activation': grid_search.best_params_['activation'],
        'learning_rate_init': grid_search.best_params_['learning_rate_init'],
        'max_iter': grid_search.best_params_['max_iter'],
        'MSE': mse,
        'MAPE': mape,
        'R2': r2
    })

    # Sauvegarder le modèle pour chaque colonne dans le dossier spécifié
    model_path = os.path.join(output_dir, f'{col}_best_model_ANN.joblib')  # Sauvegarder dans le bon dossier
    dump(best_model, model_path)
    print(f"Modèle pour la variable '{col}' enregistré sous {model_path}")

    # Afficher les résultats pour chaque colonne
    print(f"Target Column '{col}'")
    print("Best Parameters:", grid_search.best_params_)
    print(f"MSE: {mse}, MAPE: {mape}, R²: {r2}")
    print("-" * 40)

# Créer un DataFrame pour résumer les résultats
params_ANN = pd.DataFrame(best_params_list)

# Afficher le tableau des résultats
print(params_ANN)

# Créer un DataFrame pour les prédictions
Y_pred_ANN = pd.DataFrame()
for column, model in best_models.items():
    Y_pred_ANN[column] = model.predict(X_test_transformed[:k, :])

# Afficher les prédictions (si nécessaire)
# print(Y_pred_ANN)

Modèle pour la variable 'CR' enregistré sous Best_models/X_L/ANN\CR_best_model_ANN.joblib
Target Column 'CR'
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.01, 'max_iter': 500}
MSE: 0.005470027392749603, MAPE: 0.10479160324567498, R²: 0.7100426374999715
----------------------------------------
Modèle pour la variable 'BAE' enregistré sous Best_models/X_L/ANN\BAE_best_model_ANN.joblib
Target Column 'BAE'
Best Parameters: {'activation': 'tanh', 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.01, 'max_iter': 500}
MSE: 2.015755500063664, MAPE: 0.374247597886475, R²: 0.7494276110379976
----------------------------------------
  Target Column hidden_layer_sizes activation  learning_rate_init  max_iter  \
0            CR           (50, 50)       relu                0.01       500   
1           BAE           (50, 50)       tanh                0.01       500   

        MSE      MAPE        R2  
0  0.005470  0.104792  0.710043  
1  2.015